# Google Colab Connect

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Path till IR 

# path till IR final 
my_path = '/content/drive/MyDrive/SUB/PROJECTS/IR_final/datasets/'
dataset = 'Intre/'
dataset_name = 'instre'
# I<O<P<S
take_path = my_path + dataset
save_path = '/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/features/instre/'

# Import statements 

In [4]:
from sklearn.cluster import KMeans
import numpy as np
import cv2
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import math
from sklearn.metrics.pairwise import pairwise_distances
import time

from PIL import Image
import scipy.spatial.distance as d

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import json
import datetime
import copy
from PIL import Image as im 
import joblib


In [6]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [7]:
import scipy.cluster.vq as vq
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

In [8]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

# Saving and Loading function 

In [9]:
# Saving and Loading models using joblib 
def save(filename, obj):
  with open(filename, 'wb') as handle:
      joblib.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
  with open(filename, 'rb') as handle:
      return joblib.load(filename)

# Data Loaders and data class 

In [10]:
# Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomSizedCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:886: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


In [11]:
# custom class to provide the image name along with the image 
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image name
        img_name = self.imgs[index][0].split("/")[-1]
        
        # print(path)
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (img_name,))
        return tuple_with_path

       

In [16]:
train_data_dir = take_path
train_dataset = ImageFolderWithPaths(train_data_dir, transform=train_transforms) 

In [15]:
batch = 15
train_dataloader = torch.utils.data.DataLoader(train_dataset ,batch_size=batch )

# Generating Global Feature Extractor 

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# pre trained model 
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
model.to(device)
print('')

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [21]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# model.backbone.body.layer4[2].conv3.register_forward_hook(get_activation('conv3'))
model.backbone.fpn.layer_blocks[3].register_forward_hook(get_activation('conv_features'))

In [ ]:
# count = 0
# temp = []
# image_names = []

# for i,(inputs, labels, paths) in enumerate(train_dataloader):
#   model(inputs.to(device)) 
#   for f, pt in zip(activation['conv_features'], paths):
#     count = count + 1
#     if count%500 == 0:
#       print(count, "saved")

#     feat = np.sum(np.sum(f.cpu().numpy(),axis=2),axis=1)
#     print(feat.shape,pt)
#     # specify your own path to save 
#     np.save( save_path + 'fpn_' + pt , feat)
#     temp.append(feat)
#     image_names.append(pt)

In [ ]:
# # All images features appended in temp and each of following shape 
# print(temp[0].shape) , print(len(image_names))

In [ ]:
# save(save_path + dataset_name + "_all_img_features_fpn_layer.pkl",temp)
# save(save_path + dataset_name + "_all_img_names_fpn_layer.pkl",image_names)